https://www.booking.com

In [ ]:
from selenium import webdriver
from fake_useragent import UserAgent
import time
from datetime import datetime, timedelta
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import logging
from urllib3.exceptions import NewConnectionError
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
list_ville_pays = ['ss=Vienne%2C+Vienne+%28État%29%2C+Autriche', 'ss=Berlin%2C+Berlin+%28état+fédéral%29%2C+Allemagne', 'ss=Tirana%2C+Tirana+County%2C+Albanie', 'ss=Vienne%2C+Vienne+%28État%29%2C+Autriche', 'ss=Bruxelles%2C+Bruxelles%2C+Belgique', 'ss=Sarajevo%2C+Sarajevo+Canton%2C+Bosnie-Herzégovine', 'ss=Sofia%2C+Province+de+Sofia%2C+Bulgarie', 'ss=Nicosie%2C+Chypre', 'ss=Zagreb%2C+Comitat+de+Zagreb%2C+Croatie', 'ss=Copenhague%2C+Hovedstaden%2C+Danemark', 'ss=Madrid%2C+Communauté+de+Madrid%2C+Espagne', 'ss=Tallinn%2C+Harjumaa%2C+Estonie', 'ss=Helsinki%2C+Finlande-Méridionale%2C+Finlande', 'ss=Paris%2C+Île-de-France%2C+France', 'ss=Athènes%2C+Attique%2C+Grèce', 'ss=Budapest%2C+Pest%2C+Hongrie', 'ss=Dublin%2C+Comté+de+Dublin%2C+Irlande', 'ss=Reykjavik%2C+Grand+Reykjavik%2C+Islande', 'ss=Rome%2C+Latium%2C+Italie', 'ss=Pristina%2C+Pristina+County%2C+Kosovo', 'ss=Riga%2C+Vidzeme%2C+Lettonie', 'ss=Vaduz%2C+Suisse+orientale%2C+Liechtenstein', 'ss=Vilnius%2C+Vilnius+county%2C+Lituanie', 'ss=Luxembourg%2C+Luxembourg', 'ss=Skopje%2C+Macédoine+du+Nord', 'ss=La+Valette%2C+Île+de+Malte%2C+Malte', 'ss=Chişinău%2C+Moldavie', 'ss=Monte-Carlo%2C+Côte+d%27Azur%2C+Monaco', 'ss=Podgorica%2C+Podgorica+County%2C+Monténégro', 'ss=Oslo%2C+Comté+d%27Oslo%2C+Norvège', 'ss=Amsterdam%2C+Hollande-Septentrionale%2C+Pays-Bas', 'ss=Varsovie%2C+Mazovie%2C+Pologne', 'ss=Lisbonne%2C+Lisbon+Region%2C+Portugal', 'ss=Prague%2C+République+tchèque', 'ss=Bucarest%2C+Bucharest+-+Ilfov+Region%2C+Roumanie', 'ss=Londres%2C+Grand+Londres%2C+Royaume-Uni', 'ss=Belgrade%2C+Central+Serbia%2C+Serbie', 'ss=Bratislava%2C+Bratislavský+kraj%2C+Slovaquie', 'Ljubljana%2C+Osrednjeslovenska%2C+Slovénie', 'ss=Berne%2C+Canton+de+Berne%2C+Suisse', 'ss=Stockholm%2C+Comté+de+Stockholm%2C+Suède', 'ss=Kiev%2C+Ukraine', 'ss=Vatican%2C+Rome%2C+Latium%2C+Italie']

#Data frame
tab_name = []
tab_prix = []
tab_adresse = []
tab_date = []
tab_services = []
tab_ville = []
tab_note = []

In [ ]:
def get_start_end_date():
    start_date = datetime.today()
    end_date = start_date + timedelta(days=60)
    
    return start_date, end_date

In [ ]:
def get_driver(driver, start_date, end_trip, ville, start_date_str, end_trip_str, i):
    #go to the next link
    try:
        driver.get(f"https://www.booking.com/searchresults.fr.html?"+ville+"&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQGIAgGoAgO4Au7RjaoGwAIB0gIkNzQ3ZWY1ZWQtNDY5OC00ZmJhLTgxNGYtZTVhM2JlYWMzNWEx2AIF4AIB&sid=dfa66e6ef263fc9c219c2f7165d8a25f&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&offset="+str(i))
    except NewConnectionError:
        logging.error(f"Erreur de connexion pour la date "+start_date+" et la date "+end_trip)
        try:
            driver.get(f"https://www.booking.com/searchresults.fr.html?"+ville+"&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQGIAgGoAgO4Au7RjaoGwAIB0gIkNzQ3ZWY1ZWQtNDY5OC00ZmJhLTgxNGYtZTVhM2JlYWMzNWEx2AIF4AIB&sid=dfa66e6ef263fc9c219c2f7165d8a25f&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&offset="+str(i))

        except Exception as e:
            logging.error(f"impossible de scrapper cette date : "+start_date+" et "+end_trip)
            driver.quit()
    time.sleep(2)
    return driver

In [ ]:
def get_url_list(driver, start_date, end_trip, ville, start_date_str, end_trip_str):
    url_list = []
    driver = get_driver(driver, start_date, end_trip, ville, start_date_str, end_trip_str, i=0)
    
    offset = driver.find_element(By.CLASS_NAME, 'd8f77e681c').get_attribute("textContent")
    
    offset = offset.split(': ')
    offset = offset[1].split(' éta')
    
    for i in range(0, int(offset[0]), 25) :
        driver = get_driver(driver, start_date, end_trip, ville, start_date_str, end_trip_str, i)
        link_elements = driver.find_elements(By.CLASS_NAME, 'a78ca197d0')
        
        for link_element in link_elements:
            url = link_element.get_attribute("href")
            if url is not None:
                url_list.append(url)
    
    return url_list, driver

In [ ]:
def get_data(driver):
    try:
        nom_chambre = driver.find_element(By.CSS_SELECTOR, '.d2fee87262').get_attribute('textContent')
        print(nom_chambre)
        
        prix_initiale = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)').get_attribute('textContent')
        print(prix_initiale)
        
        taux_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
        print(taux_reduction) 
        
        prix_apres_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
        print(prix_apres_reduction)
        
        emplacement = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[5]/div[1]/div[1]/div[1]/div/div[3]/div[11]/p/a').get_attribute('data-atlas-latlng')
        print(emplacement)
        
    except NoSuchElementException as e:
        print(e)
        return
    
    time.sleep(3)

In [ ]:
def go_to_url(url_list, driver, start_date):
    for url in url_list:
        nom_chambre = ''
        prix_initiale = ''
        try:
            driver.get(url)
        except Exception as e:
            logging.error("impossible de se connecter")
            logging.error("reessaie...")
            try:
                driver.get(url)
            except Exception as e:
                logging.error("pas de connexion frero")
                continue
        print(url)
        # get all necessary data from the pages 
        
        try:
            nom_chambre = driver.find_element(By.CSS_SELECTOR, '.d2fee87262').get_attribute('textContent')
            print(f"nom hotel : "+ nom_chambre)
            tab_name.append(nom_chambre)
        except Exception as e:
            tab_name.append('')
        
        try :
            prix_initiale = driver.find_element(By.CLASS_NAME, 'prco-valign-middle-helper').get_attribute('textContent')
            print(f"prix : "+prix_initiale)
            tab_prix.append(prix_initiale)
        except Exception as e:
            tab_prix.append('')
        
        try :
            emplacement = driver.find_element(By.CLASS_NAME, 'hp_address_subtitle js-hp_address_subtitle jq_tooltip').get_attribute('textContent')
            print(f"adresse : "+ emplacement)
            tab_adresse.append(emplacement)
        except Exception as e:
            tab_adresse.append('')
            
        try :
            note = driver.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[10]/div/div[3]/div/div[2]/div/button/div/div/div[1]').get_attribute('textContent')
            print(f"notes : "+ note)
            print(note)
            tab_note.append(note)
        except Exception as e:
            tab_note.append('')
            print('no notes')
            
        try :
            tab_serv = []
            services = driver.find_elements(By.CLASS_NAME, 'a5a5a75131')
            for service in services:
                serv = service.get_attribute('textContent')
                tab_serv.append(serv)
            tab_services.append(tab_serv)
        except Exception:
            tab_services.append('')
            
            
        tab_date.append(start_date)
        time.sleep(2)
    
    

In [ ]:

ua = UserAgent()
user_agent = ua.random

options = webdriver.FirefoxOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument('--headless')


#change de ville a chaque 2 mois écoulé
for ville in list_ville_pays:
  capitale = ville.split('%')
  capitale = capitale[0][3:]
  
  #récupère le reservations d'hotels sur une durée de 1 jour
  start_date, end_date = get_start_end_date()
  while start_date < end_date:
  
    driver = webdriver.Firefox(options=options)
    
    #------------
    print("new date")
    end_trip = start_date + timedelta(days=1)
    start_date_str = str(start_date).split(' ')
    end_trip_str = str(end_trip).split(' ')
    #------------
    
    # get a list of url
    url_list, driver = get_url_list(driver, start_date, end_trip, ville, start_date_str, end_trip_str)
    #go to the matching url 
    go_to_url(url_list, driver, start_date)  
    print(tab_name)
    
    #ecrit le resultat dans un csv
    dict = {
      'Nom' :  tab_name,
      'Adresse' :  tab_adresse,
      'Prix intiale' : tab_prix,
      'Date' : start_date,
      'Ville' : capitale,
      'Note' : tab_note,
      'Services' : tab_services
    }
    
    hotels = pd.DataFrame(data=dict)
    hotels.to_csv(f'../hotels_'+ start_date +'.csv')
    print('the csv has been created')
    #passe a la date suivante
    start_date = end_trip
    driver.quit()
    time.sleep(3)
    